In [80]:
import utils
from electionutils import ElectionUtils

sample_orders=1000

## Init

In [81]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")
sys.path.append("../es/")

import random
myseed = 13
random.seed(myseed)

import es.data_config
import es.runner.scenario_runner
from es.runner.view_models import show_cube_grid, SpectrumViewModel
import es.elections
import es.utils
from es.data.goodreads_data import *
from bokeh.io import output_notebook
output_notebook()
%matplotlib inline

pd.set_option('display.max_rows', 40)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [82]:
%%time
idata = GoodreadsData(es.data_config.data_goodreads_parquet)
iframes = es.elections.InstacartDataFrames.from_data(idata)
ut = es.electionutils.ElectionUtils(iframes, db_filename=":memory:")

reading instacart data
reading DataFrame order_products__all from parquet: /Users/gregaw/code/ddr/esearch/es/../in/goodreads_parquet/order_products__all.parquet
order_products__all contains 4122111 items
WARN: Only products with min_popularity=20 considered (69 approvals filtered from 4122111 to 4122042)
reading DataFrame orders from parquet: /Users/gregaw/code/ddr/esearch/es/../in/goodreads_parquet/orders.parquet
orders contains 53406 items
reading DataFrame products from parquet: /Users/gregaw/code/ddr/esearch/es/../in/goodreads_parquet/products.parquet
products contains 10000 items
reading DataFrame departments from parquet: /Users/gregaw/code/ddr/esearch/es/../in/goodreads_parquet/departments.parquet
departments contains 1 items
reading DataFrame aisles from parquet: /Users/gregaw/code/ddr/esearch/es/../in/goodreads_parquet/aisles.parquet
aisles contains 1 items
creating db with 10000 products
CPU times: user 914 ms, sys: 269 ms, total: 1.18 s
Wall time: 1.09 s


## Parameters

In [83]:
PARAMS = es.utils.dotdict(dict(
        exp_name='tmp',
        k=10,
        searchterms=['^Divergent$'],
        filtering=dict(sample_orders=sample_orders),
        rules=[
            "HUV_0", 
            "sa.HUV_0", 
#             "HUV_1", "sa.HUV_1", 
#             "HUV_2", "sa.HUV_2", 
#             "HUV_3", "sa.HUV_3", 
        ], # prefix sa. means 'use simulated annealing'
        anneal_settings= {'tmax': 8100.0, 'tmin': 2.6, 'steps': 2000, 'updates': 10}, # anneal_settings={'minutes': 0.5},
        scorer_metrics=['tfidfto2'],
        scorer_funcs = [es.scorers.AgentSpreadScoringStrategy.from_found_df_sorted_nosearchterm]
    ))

## Runner

In [84]:
%%time
runner = es.runner.scenario_runner.ScenarioRunner(iframes, PARAMS)
exp_folder = runner.run(verbose=1)

WARN: limited the frame by sampling 1000 random orders to: 1000 orders, 9164 products (and 76668 order-product tuples)
loaded 9164 products and 1000 orders, (76668 ops)


 Temperature        Energy    Accept   Improve     Elapsed   Remaining


2022-06-20 07:40:37.057063
dl=4252, avgdl=8.366215626364033, products=4253 metrics_voter_count=1000, real_voter_count=1000
searching for: ^Divergent$ : ['Divergent']
----------
RUN: exp_name=tmp;ut_scorer=None
filtered iframes, products: 4252, orders:  167
processing RULE: HUV_0
Using algo/version: g/4
Using optimised HUV_0
greedy score=710.0 ([109.0, 94.0, 89.0, 72.0, 67.0, 60.0, 56.0, 55.0, 55.0, 53.0])
processing RULE: sa.HUV_0
Using algo/version: sa/2
first state= [580, 1066, 1205, 1521, 1534, 1752, 1844, 1888, 2121, 2381]
annealing with: {'tmax': 8100.0, 'tmin': 2.6, 'steps': 2000, 'updates': 10}@ 07:40:37
annealing done @ 07:40:37
annealing score=243.0
--------------------
RUN: exp_name=tmp;ut_scorer=None, search term: ^Divergent$


rank                                     HUV_0  HUV_0_id  \
0     1                          The Hunger Games         1   
1     2                             Catching Fire        17   
2     3                                 Insurgent        69   
3     4                                Mockingjay        20   
4     5  Harry Potter and the Philosopher's Stone         2   
5     6                    The Fault in Our Stars         6   
6     7      Harry Potter and the Deathly Hallows        25   
7     8       Harry Potter and the Goblet of Fire        24   
8     9  Harry Potter and the Prisoner of Azkaban        18   
9    10   Harry Potter and the Chamber of Secrets        23   

                                            sa.HUV_0  sa.HUV_0_id  
0  Het Achterhuis: Dagboekbrieven 14 juni 1942 - ...           15  
1                                          Gone Girl           30  
2                                        Paper Towns           88  
3                                          Allegiant          105  
4                                      The Lost Hero          163  
5                                  The Titan's Curse          214  
6                                       Bared to You          218  
7                                               None          324  
8                                          Graceling          355  
9                                  What Alice Forgot          438

product_id  order_count_found       product_name  order_count   tf  \
11           12                167          Divergent          167  167   
68           69                 89          Insurgent          109   89   
104         105                 52          Allegiant           62   52   
90           91                 47    The Maze Runner           69   47   
16           17                 94      Catching Fire          242   94   
257         259                 34  The Scorch Trials           47   34   
50           51                 47      City of Bones           85   47   
165         166                 26            Matched           34   26   
148         149                 33          If I Stay           51   33   
0             1                109   The Hunger Games          377  109   

          idf    idfto2       tfidf    tfidfto2      bm25   ratioIn  
11   1.789761  3.007279  298.890165  502.215527  0.828699  1.000000  
68   2.216407  3.909862  197.260258  347.977691  0.589410  0.816514  
104  2.780621  5.532269  144.592286  287.677989  0.448570  0.838710  
90   2.673649  5.179924  125.661492  243.456415  0.391867  0.681159  
16   1.418818  2.393690  133.368850  225.006877  0.396529  0.388430  
257  3.057608  6.560038  103.958661  223.041278  0.328629  0.723404  
50   2.465104  4.556235  115.859889  214.143038  0.361301  0.552941  
165  3.381395  8.005954   87.916264  208.154799  0.280279  0.764706  
148  2.975930  6.238559   98.205678  205.872438  0.310770  0.647059  
0    0.975510  1.822337  106.330600  198.634744  0.311509  0.289125

...

greedy score=2347.360589087542 ([347.9776907182128, 287.6779888368663, 243.4564147617386, 225.00687679200104, 223.0412780393286, 214.14303834276248, 208.15479944512595, 205.8724377552666, 198.63474408068583, 193.39532031555345])
processing RULE: sa.HUV_0
Using algo/version: sa/2
first state= [78, 205, 208, 280, 1353, 2035, 2597, 3036, 3345, 3503]
annealing with: {'tmax': 8100.0, 'tmin': 2.6, 'steps': 2000, 'updates': 10}@ 07:40:37


annealing done @ 07:40:37
annealing score=1751.6830426429249
--------------------
RUN: exp_name=tmp;ut_scorer=AgentSpread(tfidfto2), search term: ^Divergent$


rank              HUV_0  HUV_0_id              sa.HUV_0  sa.HUV_0_id
0     1          Insurgent        69         City of Bones           51
1     2          Allegiant       105         Breaking Dawn           56
2     3    The Maze Runner        91              The Host           73
3     4      Catching Fire        17       The Maze Runner           91
4     5  The Scorch Trials       259             Allegiant          105
5     6      City of Bones        51                Legend          327
6     7            Matched       166        The Kill Order          946
7     8          If I Stay       149  The Serpent's Shadow         1359
8     9   The Hunger Games         1                  None         2420
9    10      The Selection       206                  None         5530

product_id  order_count_found       product_name  order_count   tf  \
11           12                167          Divergent          167  167   
68           69                 89          Insurgent          109   89   
104         105                 52          Allegiant           62   52   
90           91                 47    The Maze Runner           69   47   
16           17                 94      Catching Fire          242   94   
257         259                 34  The Scorch Trials           47   34   
50           51                 47      City of Bones           85   47   
165         166                 26            Matched           34   26   
148         149                 33          If I Stay           51   33   
0             1                109   The Hunger Games          377  109   

          idf    idfto2       tfidf    tfidfto2      bm25   ratioIn  
11   1.789761  3.007279  298.890165  502.215527  0.828699  1.000000  
68   2.216407  3.909862  197.260258  347.977691  0.589410  0.816514  
104  2.780621  5.532269  144.592286  287.677989  0.448570  0.838710  
90   2.673649  5.179924  125.661492  243.456415  0.391867  0.681159  
16   1.418818  2.393690  133.368850  225.006877  0.396529  0.388430  
257  3.057608  6.560038  103.958661  223.041278  0.328629  0.723404  
50   2.465104  4.556235  115.859889  214.143038  0.361301  0.552941  
165  3.381395  8.005954   87.916264  208.154799  0.280279  0.764706  
148  2.975930  6.238559   98.205678  205.872438  0.310770  0.647059  
0    0.975510  1.822337  106.330600  198.634744  0.311509  0.289125

saved election results to results/tmp/exp=tmp,search=^Divergent$,win=10,sample_orders=1000.csv
saved tfidf results to results/tmp/exp=tmp,search=^Divergent$,win=10,sample_orders=1000_TFIDF.tsv
Results saved to 'results/tmp'
CPU times: user 535 ms, sys: 39.2 ms, total: 574 ms
Wall time: 582 ms


## Visualisation

In [85]:
%%time
es.runner.view_models.show_spectrum_grid(SpectrumViewModel(), PARAMS, exp_folder, plot_side=500, plot_height=50, max_div_value=150, verbose=0)

{'exp_name': 'tmp', 'k': 10, 'searchterms': ['^Divergent$'], 'filtering': {'sample_orders': 1000}, 'rules': ['HUV_0', 'sa.HUV_0'], 'anneal_settings': {'tmax': 8100.0, 'tmin': 2.6, 'steps': 2000, 'updates': 10}, 'scorer_metrics': ['tfidfto2'], 'scorer_funcs': [<function AgentSpreadScoringStrategy.from_found_df_sorted_nosearchterm at 0x7fb0e9130680>]}
results/tmp/exp=tmp,search=^Divergent$,win=10,sample_orders=1000.csv
(0) exp_name=tmp;ut_scorer=None


/Users/gregaw/code/ddr/esearch/es/runner/view_models.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'] = df['product_id'].apply(lambda x: id2coord(x))


(1) exp_name=tmp;ut_scorer=AgentSpread(tfidfto2)
diversity metrics:  (ar): average rank


CPU times: user 539 ms, sys: 20.9 ms, total: 560 ms
Wall time: 561 ms


In [90]:
# for name in utils.GoodreadsUtils.TITLES_MIXED:
# for name in ['Five on a Treasure Island']:
#     found = ut.db.query(f"select * from product where product_name like '%{name}%'").product_name.tolist()
#     if len(found) > 1:
#         print(f">{name}<", found)
#     elif len(found) == 1:
#         print(f">{name}<")

name = 'Five on a Treasure Island'
ut.db.query(f"select * from product where product_name like '%{name}%'")

product_id                product_name main_genre  aisle_id  department_id  \
0        6359  Five on a Treasure Island          NA         1              1   

     aisle department  order_count  
0  default    default        129.0